# Multi-Agent Systems

#### Architecture
Controller Agent – assigns tasks

Worker Agents – specialists (e.g., DataAgent, CodeAgent, ResearchAgent)

Shared Memory – database, in-memory store, or even natural language

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END


In [2]:
from typing import TypedDict

class CodeState(TypedDict):
    user_input: str
    code: str
    review: str


Code Generator Agent

In [3]:
def code_agent(state: CodeState) -> CodeState:
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful AI that writes Python code."),
        ("human", "Write code for the following task: {input}")
    ])
    
    chain = prompt | ChatOpenAI()
    result = chain.invoke({"input": state["user_input"]})
    
    return {**state, "code": result.content}


QA Agent

In [4]:
def qa_agent(state: CodeState) -> CodeState:
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a code reviewer. Give suggestions or approve."),
        ("human", "Review this code:\n{code}")
    ])
    
    chain = prompt | ChatOpenAI()
    result = chain.invoke({"code": state["code"]})
    
    return {**state, "review": result.content}


Build the LangGraph Workflow

In [5]:
builder = StateGraph(CodeState)

# Add steps
builder.add_node("coder", code_agent)
builder.add_node("qa", qa_agent)

# Define flow
builder.set_entry_point("coder")
builder.add_edge("coder", "qa")
builder.add_edge("qa", END)

# Compile
graph = builder.compile()


Run the Graph

In [7]:
result = graph.invoke({"user_input": "create a python file to create aws ec2 instance using boto3"})

print("🧾 Generated Code:\n", result["code"])
print("\n🔍 QA Feedback:\n", result["review"])


🧾 Generated Code:
 Sure! Here is an example code snippet that creates an AWS EC2 instance using the boto3 library in Python:

```python
import boto3

# Configure the AWS credentials
aws_access_key_id = 'YOUR_ACCESS_KEY_ID'
aws_secret_access_key = 'YOUR_SECRET_ACCESS_KEY'
region = 'us-west-2'

# Create a new EC2 instance
ec2 = boto3.resource('ec2',
                     aws_access_key_id=aws_access_key_id,
                     aws_secret_access_key=aws_secret_access_key,
                     region_name=region)

instance = ec2.create_instances(
    ImageId='ami-0c55b159cbfafe1f0',
    MinCount=1,
    MaxCount=1,
    InstanceType='t2.micro',
    KeyName='YOUR_KEY_PAIR_NAME',
    SecurityGroupIds=['YOUR_SECURITY_GROUP_ID'],
    SubnetId='YOUR_SUBNET_ID'
)

print("Instance created:", instance[0].id)
```

Make sure to replace the placeholders `YOUR_ACCESS_KEY_ID`, `YOUR_SECRET_ACCESS_KEY`, `YOUR_KEY_PAIR_NAME`, `YOUR_SECURITY_GROUP_ID`, and `YOUR_SUBNET_ID` with your actual AWS credentials a

### Next Ideas:

Add a FixAgent to iterate based on review

Log results to a database

Use tool-using agents (e.g., one that runs the code with Python REPL)

Add a human-in-the-loop UI for approvals

